Folositi urmatoarele seturi de date:
1. [CPU Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware); excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".
1. [Boston Housing](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/)
1. [Wisconsin Breast Cancer](http://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html); cautati in panelul din stanga Wisconsin Breast Cancer si urmati pasii din "My personal Notes"
1. [Communities and Crime](http://archive.ics.uci.edu/ml/datasets/communities+and+crime); stergeti primele 5 dimensiuni si trasaturile cu missing values.

In [820]:
import numpy as np
print(f'NumPy version: {np.__version__}')

import pandas as pd
print(f'Pandas version: {pd.__version__}')

import sklearn as sk
from sklearn.impute import SimpleImputer
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
print(f'Sklearn version: {sk.__version__}')

#NumPy version: 1.20.1
#Pandas version: 1.2.3
#Sklearn version: 0.24.1

NumPy version: 1.20.1
Pandas version: 1.2.3
Sklearn version: 0.24.1


In [808]:
machine: pd.core.frame.DataFrame = pd.read_csv("Data/machine.data", header=None)
x_machine: np.ndarray = machine.iloc[:, 2:8].values
y_machine: np.ndarray = machine.iloc[:, 8].values
y_machine = y_machine.reshape(y_machine.shape[0])

In [809]:
housing: pd.core.frame.DataFrame = pd.read_csv("Data/housing.data", delim_whitespace=True, header=None)
x_housing: np.ndarray = housing.iloc[:, :-1].values
y_housing: np.ndarray = housing.iloc[:, -1].values
y_housing = y_housing.reshape(y_housing.shape[0])

In [810]:
r_wpbc: pd.core.frame.DataFrame = pd.read_csv("Data/r_wpbc.data", header=None)
x_r_wpbc: np.ndarray = r_wpbc.iloc[:, 1:].values
y_r_wpbc: np.ndarray = r_wpbc.iloc[:, 1].values
y_r_wpbc = y_r_wpbc.reshape(y_r_wpbc.shape[0])

In [811]:
communities: pd.core.frame.DataFrame = pd.read_csv("Data/communities.data", header=None)
communities = communities.replace('?', np.nan)    
x_communities: np.ndarray = communities.iloc[:, 1:].drop([3], axis=1).values
y_communities: np.ndarray = communities.iloc[:, 0].values
y_communities = y_communities.reshape(y_communities.shape[0])

imp:sk.impute._base.SimpleImputer = SimpleImputer(missing_values = np.nan, strategy="median")
x_communities = imp.fit_transform(x_communities)

Pentru fiecare set de date aplicati minim 5 modele de regresie din scikit learn. Pentru fiecare raportati: mean absolute error, mean squared error, median absolute error - a se vedea sklearn.metrics - folosind 5 fold cross validation. Valorile hiperparametrilor trebuie cautate cu grid search (cv=3) si random search (n_iter dat de voi). Metrica folosita pentru optimizarea hiperparametrilor va fi mean squared error. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare; indicatie: puteti folosi metoda cross_validate cu parametrul return_train_score=True, iar ca model un obiect de tip GridSearchCV sau RandomizedSearchCV.

In [812]:
def show_metrics_regression(reg, parameters: dict, x: np.ndarray, y: np.ndarray) -> pd.core.frame.DataFrame:
    """
    Shows the metrics('mean_absolute_error', 'mean_squared_error', 'median_absolute_error') of a regressor.
    
    Args:
        reg: a regressor
        parameters:a dictionary containning the hiperparameters
        x: np.array containning the dataset information
        y: np.array containning the classification of the data
        
    Returns:
        a pandas dataframe with the metrics of a regressor
    """
    gridsrc = GridSearchCV(estimator=reg, 
            param_grid=parameters, cv=3, n_jobs=-1, return_train_score=True)
    randsrc = RandomizedSearchCV(estimator=reg,
            param_distributions=parameters, n_iter=15, n_jobs=-1, return_train_score=True)
    
    scoring = ['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error']
    
    scores1 = cross_validate(gridsrc, x, y, cv=5, scoring=scoring, return_train_score=True)
    scores2 = cross_validate(randsrc, x, y, cv=5, scoring=scoring, return_train_score=True)
    
    df1 = pd.DataFrame(data={'train_neg_mean_absolute_error': scores1['train_neg_mean_absolute_error'],
                            'train_neg_mean_squared_error': scores1['train_neg_mean_squared_error'],
                            'train_neg_median_absolute_error': scores1['train_neg_median_absolute_error'],
                            'test_neg_mean_absolute_error': scores1['test_neg_mean_absolute_error'],
                            'test_neg_mean_squared_error': scores1['test_neg_mean_squared_error'],
                            'test_neg_median_absolute_error':scores1['test_neg_median_absolute_error']
                           })
    df2 = pd.DataFrame(data={'train_neg_mean_absolute_error': scores2['train_neg_mean_absolute_error'],
                            'train_neg_mean_squared_error': scores2['train_neg_mean_squared_error'],
                            'train_neg_median_absolute_error': scores2['train_neg_median_absolute_error'],
                            'test_neg_mean_absolute_error':scores2['test_neg_mean_absolute_error'],
                            'test_neg_mean_squared_error':scores2['test_neg_mean_squared_error'],
                            'test_neg_median_absolute_error': scores2['test_neg_median_absolute_error']
                           })
    
    result = pd.DataFrame([df1.mean(), df2.mean()])
    result.insert(0, 'Model_name', [reg, reg])
    result.insert(1, 'Search_strategy', ['GridSearchCV', 'RandomizedSearchCV'])
    return result

In [813]:
parameters_SGDRegressor:dict = {
    'max_iter':[10000],
    'loss': ['squared_loss','huber','epsilon_insensitive','squared_epsilon_insensitive'],
    'penalty' : ['l1', 'l2'],
    'alpha' : [0.001, 0.01, 0.1, 1]
}
parameters_RandomForestRegressor:dict = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [10, 50, 100],
    'n_jobs':[-1]
}
parameters_Lasso:dict = {
    'alpha':[0.01,0.1,1],
    'tol':[0.0001,0.001,0.01,0.1],
    'selection':['cyclic','random']
}
parameters_MLPRegressor:dict = {
    'max_iter':[10000],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.01, 0.1, 1],
    'tol':[0.0001, 0.001, 0.01]
}

parameters_Ridge:dict = {
    'alpha': [0.001, 0.01, 0.1, 1],
    "fit_intercept": [True, False],
    "solver": ['svd', 'lsqr', 'sag', 'saga']
}

In [814]:
def dataset_regression(name:str, x:np.ndarray, y:np.ndarray):
    df1 = show_metrics_regression(SGDRegressor(), parameters_SGDRegressor, x, y)
    df2 = show_metrics_regression(RandomForestRegressor(),parameters_RandomForestRegressor, x, y)
    df3 = show_metrics_regression(Lasso(), parameters_Lasso, x, y)
    df4 = show_metrics_regression(MLPRegressor(), parameters_MLPRegressor, x, y)
    df5 = show_metrics_regression(Ridge(), parameters_Ridge, x, y)

    df = pd.concat([df1, df2, df3, df4, df5], axis=0, ignore_index=True)
    df.columns.name = name

    return df

Rezultatele vor fi trecute intr-un dataframe. Intr-o stare intermediara, valorile vor fi calculate cu semnul minus: din motive de implementare, biblioteca sklearn transforma scorurile in numere negative; a se vedea imaginea de mai jos:

![intermediate report](./images/cpu_intermediate_blurred.png)


Valorile vor fi aduse la interval pozitiv, apoi vor fi marcate cele maxime si minime; orientativ, se poate folosi imaginea de mai jos, reprezentand dataframe afisat in notebook; puteti folosi alte variante de styling pe dataframe precum la https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#.  

Se va crea un raport final in format HTML sau PDF - fisier(e) separat(e). Raportul trebuie sa contina minimal: numele setului de date si obiectul dataframe; preferabil sa se pastreze marcajul de culori realizat in notebook.

![report](./images/cpu_results_blurred.png)

In [815]:
def highlight_max(s)->list:    
    """
    Highlight the maximum in a dataframe red for maximum and green for minimum.
    """
    
    max_val:float = s.max()
    min_val:float = s.min()
    return ['background-color: #ff6666' if v==max_val and type(v)
            else 'background-color: #bdfcc2'if v==min_val else '' for v in s]

def finishing(df:pd.core.frame.DataFrame)->pd.core.frame.DataFrame:
    """
    Args:
        df:a dataframe
    
    Returns:
        a dataframe with positive numbers and highlights the maximum and minimum.
    """
    aux = df.columns.name
    df.iloc[:, 2:] = df.iloc[:, 2:].abs()

    df.columns = ['Model_name', 'Search_strategy',
                  'train_mean_absolute_error', 'train_mean_squared_error',
                  'train_median_absolute_error', 'test_mean_absolute_error',
                  'test_mean_squared_error', 'test_median_absolute_error']

    df = df.style.apply(highlight_max, subset=df.columns[2:])
    df.columns.name = aux
    return df

In [816]:
df_machine:pd.core.frame.DataFrame = dataset_regression('Machine_Dataset',x_machine,y_machine)
display(df_machine)
display(finishing(df_machine))

Machine_Dataset,Model_name,Search_strategy,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error
0,SGDRegressor(),GridSearchCV,-5885.093607,-1.813542e+08,-3954.861472,-5768.977054,-1.425822e+08,-3941.358170
1,SGDRegressor(),RandomizedSearchCV,-9540.438804,-3.277660e+08,-6154.653394,-7907.508644,-1.674969e+08,-5393.851714
2,RandomForestRegressor(),GridSearchCV,-17.703193,-1.468325e+03,-7.323613,-34.396981,-6.870189e+03,-16.004778
3,RandomForestRegressor(),RandomizedSearchCV,-17.798161,-1.520029e+03,-7.393676,-36.727278,-8.244080e+03,-17.905227
4,Lasso(),GridSearchCV,-37.031790,-3.327090e+03,-24.383527,-42.818305,-6.234353e+03,-27.634102
5,Lasso(),RandomizedSearchCV,-36.677291,-3.356488e+03,-24.592035,-43.930991,-6.643058e+03,-26.542658
6,MLPRegressor(),GridSearchCV,-343.640039,-3.161751e+05,-230.687330,-340.938069,-3.962901e+05,-179.499589
7,MLPRegressor(),RandomizedSearchCV,-104.102868,-5.709457e+04,-58.245704,-117.171860,-5.625745e+04,-78.734495
8,Ridge(),GridSearchCV,-38.232255,-3.833877e+03,-25.170574,-43.683220,-6.080172e+03,-28.665729
9,Ridge(),RandomizedSearchCV,-39.359822,-4.201911e+03,-26.012799,-43.652248,-6.129041e+03,-28.422177


Machine_Dataset,Model_name,Search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error
0,SGDRegressor(),GridSearchCV,5885.093607,181354249.981082,3954.861472,5768.977054,142582161.274290,3941.358170
1,SGDRegressor(),RandomizedSearchCV,9540.438804,327766004.181475,6154.653394,7907.508644,167496870.229298,5393.851714
2,RandomForestRegressor(),GridSearchCV,17.703193,1468.324596,7.323613,34.396981,6870.189350,16.004778
3,RandomForestRegressor(),RandomizedSearchCV,17.798161,1520.029386,7.393676,36.727278,8244.080125,17.905227
4,Lasso(),GridSearchCV,37.031790,3327.089724,24.383527,42.818305,6234.352528,27.634102
5,Lasso(),RandomizedSearchCV,36.677291,3356.488468,24.592035,43.930991,6643.058040,26.542658
6,MLPRegressor(),GridSearchCV,343.640039,316175.147476,230.687330,340.938069,396290.085912,179.499589
7,MLPRegressor(),RandomizedSearchCV,104.102868,57094.569481,58.245704,117.171860,56257.449458,78.734495
8,Ridge(),GridSearchCV,38.232255,3833.877448,25.170574,43.683220,6080.171557,28.665729
9,Ridge(),RandomizedSearchCV,39.359822,4201.911101,26.012799,43.652248,6129.040580,28.422177


In [817]:
df_housing:pd.core.frame.DataFrame = dataset_regression('Housing_Dataset',x_housing,y_housing)
display(df_housing)
display(finishing(df_housing))

Housing_Dataset,Model_name,Search_strategy,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error
0,SGDRegressor(),GridSearchCV,-11.457279,-181.670415,-10.956268,-13.379730,-228.960285,-12.903078
1,SGDRegressor(),RandomizedSearchCV,-9.778328,-151.994689,-8.198012,-10.338941,-167.563971,-8.439105
2,RandomForestRegressor(),GridSearchCV,-1.233586,-3.144196,-0.893302,-3.159174,-23.407905,-2.211349
3,RandomForestRegressor(),RandomizedSearchCV,-1.255624,-3.098378,-0.893906,-3.090621,-22.220433,-2.125346
4,Lasso(),GridSearchCV,-3.404630,-24.210671,-2.441830,-4.520350,-44.162859,-3.189411
5,Lasso(),RandomizedSearchCV,-3.413915,-24.135130,-2.441316,-4.527627,-43.405540,-3.272644
6,MLPRegressor(),GridSearchCV,-4.953644,-43.595120,-4.201025,-5.684218,-59.615954,-4.520040
7,MLPRegressor(),RandomizedSearchCV,-4.460622,-37.568302,-3.517934,-5.918385,-63.513266,-4.759539
8,Ridge(),GridSearchCV,-3.576946,-25.736149,-2.567906,-3.892214,-30.705811,-2.863165
9,Ridge(),RandomizedSearchCV,-3.391719,-23.148354,-2.491723,-4.077441,-37.789718,-2.754568


Housing_Dataset,Model_name,Search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error
0,SGDRegressor(),GridSearchCV,11.457279,181.670415,10.956268,13.379730,228.960285,12.903078
1,SGDRegressor(),RandomizedSearchCV,9.778328,151.994689,8.198012,10.338941,167.563971,8.439105
2,RandomForestRegressor(),GridSearchCV,1.233586,3.144196,0.893302,3.159174,23.407905,2.211349
3,RandomForestRegressor(),RandomizedSearchCV,1.255624,3.098378,0.893906,3.090621,22.220433,2.125346
4,Lasso(),GridSearchCV,3.404630,24.210671,2.441830,4.520350,44.162859,3.189411
5,Lasso(),RandomizedSearchCV,3.413915,24.135130,2.441316,4.527627,43.405540,3.272644
6,MLPRegressor(),GridSearchCV,4.953644,43.595120,4.201025,5.684218,59.615954,4.520040
7,MLPRegressor(),RandomizedSearchCV,4.460622,37.568302,3.517934,5.918385,63.513266,4.759539
8,Ridge(),GridSearchCV,3.576946,25.736149,2.567906,3.892214,30.705811,2.863165
9,Ridge(),RandomizedSearchCV,3.391719,23.148354,2.491723,4.077441,37.789718,2.754568


In [818]:
df_r_wpbc:pd.core.frame.DataFrame = dataset_regression('WPBC_Dataset',x_r_wpbc,y_r_wpbc)
display(df_r_wpbc)
display(finishing(df_r_wpbc))

WPBC_Dataset,Model_name,Search_strategy,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error
0,SGDRegressor(),GridSearchCV,-113.219908,-1.952286e+04,-103.233533,-119.777895,-2.171267e+04,-107.598690
1,SGDRegressor(),RandomizedSearchCV,-104.682131,-1.565307e+04,-100.538973,-111.726639,-1.779116e+04,-102.967984
2,RandomForestRegressor(),GridSearchCV,-0.091155,-5.931311e-02,-0.041314,-0.149932,-1.072149e-01,-0.071898
3,RandomForestRegressor(),RandomizedSearchCV,-0.086298,-5.101598e-02,-0.040410,-0.135142,-9.521870e-02,-0.069752
4,Lasso(),GridSearchCV,-0.000417,-2.673728e-07,-0.000368,-0.000430,-2.806061e-07,-0.000379
5,Lasso(),RandomizedSearchCV,-0.000886,-2.576392e-06,-0.000767,-0.000956,-3.123603e-06,-0.000869
6,MLPRegressor(),GridSearchCV,-33.446796,-2.521527e+03,-30.904421,-34.904936,-2.748067e+03,-29.656743
7,MLPRegressor(),RandomizedSearchCV,-26.158862,-1.379456e+03,-25.280517,-25.622695,-1.343819e+03,-23.695473
8,Ridge(),GridSearchCV,-0.000050,-5.082138e-09,-0.000036,-0.000064,-9.226372e-09,-0.000046
9,Ridge(),RandomizedSearchCV,-0.000050,-5.087074e-09,-0.000036,-0.000063,-9.184931e-09,-0.000046


WPBC_Dataset,Model_name,Search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error
0,SGDRegressor(),GridSearchCV,113.219908,19522.856947,103.233533,119.777895,21712.669487,107.598690
1,SGDRegressor(),RandomizedSearchCV,104.682131,15653.071087,100.538973,111.726639,17791.156511,102.967984
2,RandomForestRegressor(),GridSearchCV,0.091155,0.059313,0.041314,0.149932,0.107215,0.071898
3,RandomForestRegressor(),RandomizedSearchCV,0.086298,0.051016,0.040410,0.135142,0.095219,0.069752
4,Lasso(),GridSearchCV,0.000417,0.000000,0.000368,0.000430,0.000000,0.000379
5,Lasso(),RandomizedSearchCV,0.000886,0.000003,0.000767,0.000956,0.000003,0.000869
6,MLPRegressor(),GridSearchCV,33.446796,2521.526795,30.904421,34.904936,2748.066568,29.656743
7,MLPRegressor(),RandomizedSearchCV,26.158862,1379.455947,25.280517,25.622695,1343.818586,23.695473
8,Ridge(),GridSearchCV,0.000050,0.000000,0.000036,0.000064,0.000000,0.000046
9,Ridge(),RandomizedSearchCV,0.000050,0.000000,0.000036,0.000063,0.000000,0.000046


In [819]:
df_communities:pd.core.frame.DataFrame = dataset_regression('Communities_Dataset',x_communities,y_communities)
display(df_communities)
display(finishing(df_communities))

Communities_Dataset,Model_name,Search_strategy,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error
0,SGDRegressor(),GridSearchCV,-137634.350408,-8.622653e+10,-140473.911413,-140461.378866,-8.812740e+10,-140475.020119
1,SGDRegressor(),RandomizedSearchCV,-35747.316367,-2.824494e+09,-36286.219621,-35500.910356,-2.808164e+09,-36287.543126
2,RandomForestRegressor(),GridSearchCV,-3.632799,-2.491163e+01,-2.573229,-8.308177,-1.252107e+02,-6.141204
3,RandomForestRegressor(),RandomizedSearchCV,-3.775504,-2.705820e+01,-2.686400,-8.325402,-1.263839e+02,-6.061256
4,Lasso(),GridSearchCV,-9.050223,-1.350846e+02,-7.148300,-9.504306,-1.491005e+02,-7.459066
5,Lasso(),RandomizedSearchCV,-9.048427,-1.350178e+02,-7.162777,-9.500737,-1.490350e+02,-7.447256
6,MLPRegressor(),GridSearchCV,-15.118910,-3.000628e+02,-14.618987,-15.296028,-3.099525e+02,-15.014231
7,MLPRegressor(),RandomizedSearchCV,-15.670648,-3.289968e+02,-15.653803,-15.950961,-3.431451e+02,-15.561572
8,Ridge(),GridSearchCV,-8.945809,-1.328236e+02,-7.025128,-9.477769,-1.493840e+02,-7.538105
9,Ridge(),RandomizedSearchCV,-8.936577,-1.320987e+02,-7.084738,-9.467082,-1.496756e+02,-7.600256


Communities_Dataset,Model_name,Search_strategy,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error
0,SGDRegressor(),GridSearchCV,137634.350408,86226528801.571869,140473.911413,140461.378866,88127400225.179596,140475.020119
1,SGDRegressor(),RandomizedSearchCV,35747.316367,2824493671.341488,36286.219621,35500.910356,2808164003.077396,36287.543126
2,RandomForestRegressor(),GridSearchCV,3.632799,24.911631,2.573229,8.308177,125.210738,6.141204
3,RandomForestRegressor(),RandomizedSearchCV,3.775504,27.058196,2.686400,8.325402,126.383922,6.061256
4,Lasso(),GridSearchCV,9.050223,135.084598,7.148300,9.504306,149.100517,7.459066
5,Lasso(),RandomizedSearchCV,9.048427,135.017764,7.162777,9.500737,149.035016,7.447256
6,MLPRegressor(),GridSearchCV,15.118910,300.062849,14.618987,15.296028,309.952458,15.014231
7,MLPRegressor(),RandomizedSearchCV,15.670648,328.996796,15.653803,15.950961,343.145084,15.561572
8,Ridge(),GridSearchCV,8.945809,132.823556,7.025128,9.477769,149.384027,7.538105
9,Ridge(),RandomizedSearchCV,8.936577,132.098713,7.084738,9.467082,149.675582,7.600256


## Random Forest Regressor
<div style="text-align: justify">
&emsp;&emsp;Pădurile aleatoare sau pădurile de decizie aleatoare sunt o metodă de învățare utilizată pentru clasificare, regresie și alte sarcini care operează prin construirea unei multitudini de arbori de decizie în etapa de antrenare și are ca rezultat clasa care are votul majoritar (pentru clasificare) sau media predicțiilor (pentru regresie) a arborilor individuali. Pădurile aleatoare corectează tendința arborilor de decizie de învățare excesivă a datelor de antrenare (overfit).
    
![random_forest_image1](./images/RandomForest.png)

&emsp;&emsp;Algoritmul de antrenament pentru pădurile aleatoare aplică tehnica generală de agregare bootstrap ("bagging") pentru arborii de învățare. Pentru un set de antrenare cu intrările $X = x_1, ..., x_n$ și ieșirile corespunzătoare $Y = y_1, ..., y_n$ prin această tehnică aplicată de $B$ ori se selectează un subset aleator din setul de date de antrenare și antrenează câte un arbore de decizie pentru acest subset. După antrenare, predicțiile pentru exemplele nevăzute $x'$ se obțin prin calculul mediei predicțiilor a arborilor individuali pentru $x'$.<br>
&emsp;&emsp;&emsp;&emsp;For $b = 1, ..., B$:<br>
       &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;1. Sample, with replacement, $n$ training examples from $X$, $Y$; call these $X_b$, $Y_b$.<br>
       &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;2. Train a classification or regression tree $f_b$ on $X_b$, $Y_b$.<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$y'=\hat{f} = {\frac {1}{B}}{\sum _{b=1}^{B}f_b(x')}$<br>
&emsp;&emsp;Folosirea metodei de agregare bootstrap conduce la performanțe mai bune deoarece reduce varianța modelului, fără a crește bias-ul. Asta înseamnă că deși predicțiile unui arbore sunt sensibile la zgomotul din subsetul său de antrenare, media predicțiilor arborilor nu este, cât timp arborii sunt independenți (antrenați pe subseturi disjuncte de date).<br>
&emsp;&emsp;Algoritmul pădurilor aleatoare diferă de metoda descrisă prin faptul că în momentul împărțirii candidaților se selectează aleator un subset de atribute ale acestora ("feature bagging"), fiecare arbore având acces la un subset aleator din datele de antrenare. Astfel crește diversitatea pădurii, cea ce conduce la predicții mai robuste.<br>

Printre hiper-parametrii utilizați în algoritmul pădurilor aleatoare sunt:
- numărul de arbori, $B$ (n_estimators) pe care algoritmul îi construiește; în general un număr mai mare de arbori crește performanța și conduce la predicții mai stabile, dar scade viteza de calcul
- numărul maxim de atribute (max_features) care să fie luate în considerare la împărțirea candidaților
- numărul minim de frunze necesar pentru a împărți un nod intern (min_sample_leaf)

https://en.wikipedia.org/wiki/Random_forest<br>
https://medium.com/@williamkoehrsen/random-forest-simple-explanation-377895a60d2d<br>
https://towardsdatascience.com/the-random-forest-algorithm-d457d499ffcd
</div>

## Ridge Regressor
<div style="text-align: justify">
&emsp;&emsp;Regresia de "creastă", numită și regularizarea Tikhonov, este o tehnică pentru analiza datelor de regresie multiple care suferă de multicoliniaritate. Atunci când apare multicoliniaritatea, estimările celor mai mici pătrate sunt imparțiale, dar variațiile lor sunt mari, astfel încât acestea pot fi departe de adevărata valoare. Prin adăugarea unui grad de părtinire la valorile estimate de regresie, Ridge reduce erorile standard. Acest lucru se face cu speranța că efectul net va fi acela de a oferi estimări mai fiabile.
    
&emsp;&emsp;Multicolinearitatea, este existența relațiilor aproape liniare între variabilele independente. De exemplu, să presupunem că cele trei ingrediente ale unui amestec sunt studiate prin includerea procentelor în care acestea apar în amestec. Aceste variabile vor avea o relație liniară (perfectă): P1 + P2 + P3 = 100. În timpul calculelor de regresie, această relație va determina o divizare cu zero, ceea ce determină, la rândul ei, întreruperea calculelor.

![ridge image](./images/Ridge.png)

&emsp;&emsp;Să presupunem că pentru o matrice $A$ și un vector $b$ vrem să găsim un vector $x$ astfel încât $Ax=b$. Abordarea standard este regresia liniară comună cu cele mai mici pătrate. Totuși, dacă nici un $x$ nu satisface ecuația sau dacă sunt mai mulți care o fac (soluția nu este unică) atunci spunem că problema este pusă greșit (ill posed). În astfel de cazuri, estimarea obișnuită a celor mai mici pătrate duce la un sistem de ecuații supradeterminat (over-fitted) sau, mai des, un sistem nedeterminat (under-fitted). Cele mai multe fenomene din lumea reală au efectul filtrelor low-pass în direcția directă(forword) unde $A$ mapează $x$ la $b$. Prin urmare, în rezolvarea problemei inverse, maparea inversă funcționează ca un filtru high-pass, care are tendința nedorită de amplificare a zgomotului (valorile proprii / valorile singulare sunt cele mai mari în maparea inversă în cazul în care au fost cele mai mici în maparea directă).
    
&emsp;&emsp;Cele mai mici pătrate încearcă să minimizeze suma reziduurilor pătrate, care pot fi scrise compact ca: 

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$||Ax-b||_2^2$


&emsp;&emsp;Pentru a avantaja o soluție particulară cu proprietățile dorite, în această minimizare poate fi inclus un termen de regularizare: 

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$||Ax-b||_2^2+||\Gamma x||_2^2$<br>
pentru matrici $\Gamma$ Tikhonov alese adecvat.<br>
&emsp;&emsp;În multe cazuri aceasta este aleasă ca fiind multiplul matricii identitate ($\Gamma = \alpha I$), avantajând soluții cu normele mai mici. Acest lucru este recunoscut sub numele de regularizare $L_2$

&emsp;&emsp;Astfel prin regularizarea Tikhonov, algoritmul Ridge, obține rezultate mai bune decât în cazul utilizării ordinare a celor mai mici pătrate.

&emsp;&emsp;Printre hiper-parametrii utilizați în algoritmul Ridge din biblioteca scikit-learn este $alpha$ (puterea regularizării), parametru cu care este înmulțită matricea identitate. Cu cât acesta este mai mare cu atât regularizarea este mai puternică.

https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Ridge_Regression.pdf<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html<br>
https://en.wikipedia.org/wiki/Tikhonov_regularization<br>
https://www.youtube.com/watch?v=Q81RR3yKn30

## Lasso Regressor
<div style="text-align: justify">
&emsp;&emsp;Regresia "lasou" este o metodă analitică de regresie care efectuează atât selecția variabilelor, cât și regularizarea, pentru a spori acuratețea predicției și interpretabilitatea modelului statistic pe care îl produce. Lasso este un tip de regresie liniară care folosește "contracția" datelor (shrinkage). Aceata presupune reducerea datelor spre un punct central, precum media. Procedura lasso încurajează modele simple, rare (adică modele cu mai puțini parametri). Acest tip particular de regresie este bine adaptat modelelor care prezintă niveluri ridicate de muticoliniaritate sau când doriți să automatizați anumite părți ale selecției modelului, cum ar fi selectarea variabilelor / eliminarea parametrilor.
    
![lasso image](./images/Lasso.png)

&emsp;&emsp;Acronimul "LASSO" reprezintă $L$east $A$bsolute $S$hrinkage and $S$election $O$perator(operator de contracție și selecție absolută minimă).<br>
&emsp;&emsp;Regresia Lasso efectuează regularizarea L1, care adaugă o penalizare egală cu valoarea absolută a magnitudinii coeficienților. Acest tip de regularizare poate duce la modele rare, cu câțiva coeficienți; Unii dintre aceștia devenind zero și putând fi eliminați din model. Penalizările mai mari generează valori ale coeficienților mai aproape de zero, ceea ce este ideal pentru producerea de modele mai simple. Pe de altă parte, regularizarea L2 (folosită, de exemplu, în regresia Ridge) nu are ca rezultat eliminarea coeficienților sau a modelelor rare. Acest lucru face Lasso mult mai ușor de interpretat decât Ridge.<br>
&emsp;&emsp;Soluțiile Lasso sunt probleme de programare patrate, care sunt cel mai bine rezolvate cu software, biblioteci specializate (cum ar fi scikit-learn). Scopul algoritmului este de a minimiza:

&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;${\sum _{i=1}^{n}{\left (  y_i-{\sum _{j}{x_{ij}\beta_j}} \right )^2}}+{\alpha}{\sum _{j=1}^{p}{|\beta_j|}}$

&emsp;&emsp;Care este același cu minimizarea sumei de pătrate cu constrângerea ${\sum _{j=1}^{p}{|\beta_j|}}\leqslant s$. Unele valori de $\beta$ sunt reduse la zero, rezultând un model de regresie mai ușor de interpretat.


&emsp;&emsp;Printre hiper-parametrii utilizați în algoritmul Lasso din biblioteca scikit-learn este $alpha$ (puterea regularizării $L1$). $alpha$ este cu alte cuvinte proporția de reducere:
- atunci când $alpha$ = 0, nu se elimină nici un parametru. Estimarea este egală cu cea găsită cu regresia liniară.
- pe măsură ce $alpha$ crește, tot mai mulți coeficienți sunt setați la zero și eliminați (teoretic, atunci când $alpha$ = $\infty$ , toți coeficienții sunt eliminați).
- pe măsură ce $alpha$ crește, bias crește.
- pe măsură ce $alpha$ scade, variance crește.<br>
&emsp;&emsp;Dacă în model este inclus un intercept, acesta este de obicei lăsat neschimbat.

https://www.statisticshowto.datasciencecentral.com/lasso-regression/<br>
https://en.wikipedia.org/wiki/Lasso_(statistics)<br>
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html<br>
https://www.slideshare.net/kaz_yos/visual-explanation-of-ridge-regression-and-lasso<br>
https://www.youtube.com/watch?v=NGf0voTMlcs<br>

## Multi-Layer Perceptron Regressor
&emsp;&emsp;Reţelele neurale multistrat — sau perceptronii multistrat, multilayer perceptrons (MLPs) — sunt folosite pentru probleme de regresie, de clasificare şi de estimare de probabilităţi condiţionate. Instruirea este supervizată. Sunt cea mai populară variantă de reţele neurale artificiale şi fac parte din clasa mai mare a reţelelor cu propagare înainte (feed-forward). <br>

&emsp;&emsp;O reţea multistrat se compune din minim trei straturi:
- strat de intrare ce preia valorile de intrare; nu are rol computaţional, nu este format din neuroni;
- cel puțin un strat ascuns, compus din neuroni;
- strat de ieşire, de asemenea compus din neuroni, produce estimări de valori care sunt apoi comparate cu ieşirile dorite;

![mlp image](./images/MLP.png)

&emsp;&emsp;Un strat ascuns este unul care nu primeşte direct intrări şi nu produce valori de ieşire. Neuronii ascunşi produc trăsături noi pe baza vectorilor de intrare, trăsături care sunt mai apoi necesare reţelei neurale pentru producerea unei estimări. Este posibil ca o reţea să aibă mai mult de un neuron în stratul de ieşire. Se consideră că instruirea e mai eficientă dacă pe lângă valorile de intrare şi pe lângă valorile calculate de un strat de neuroni se mai furnizează o valoare constantă, de regulă +1, înmulţită cu o pondere de bias. Ponderile dintre straturi precum şi aceste ponderi de bias sunt instruibile, adică se vor modifica prin procesul de învăţare.

&emsp;&emsp;Rețelele neurale cu propagare înainte precum perceptronul multistrat realizează în principal două mișcări, una înainte și una inversă. Odată ce arhitectura reţelei e fixată – numărul de straturi ascunse şi numărul de neuroni în fiecare strat precum şi funcţiile de activare – se poate trece la instruirea şi apoi utilizarea ei. Pasul de propagare înainte preia un vector de intrare $x = (x_1, . . . , x_n)^t$ şi produce modificări în starea neuronilor reţelei pornind de la intrare şi acţionând asupra succesiv straturilor $2, . . . , L −1$. Ieşirile din ultimul strat sunt
folosite pentru predicţie – regresie, estimare de probabilitate condiţionată sau clasificare. La antrenarea rețelei, în etapa de propagare înainte, semnalul trece de la stratul de intrare, prin straturile ascunse, iar rezultatul obținut în stratul de ieșire este comparat cu cel adevărat din setul de antrenare; în etapa de propagare înapoi (backpropagation), derivatele parțiale ale funcției de eroare cu respect față de diferitele ponderi și valori de bias sunt retro-propagate prin rețea. Prin efectuarea diferenței dintre predicția rețelei și valuarea așteptată se obține un gradient față de care se actualizează parametrii rețelei, aducând perceptronul multistrat mai aproape de eroarea minimă.

&emsp;&emsp;Fiecare pereche din setul de instruire $(x, d)$  va produce valoare de eroare astfel: se furnizează vectorul $x$ ca intrare în reţea şi se calculează un vector de ieşire $o$, reprezentând estimarea produsă de reţea pentru intrarea
furnizată; se foloseşte o funcţie de cost, sau de eroare, care se doreşte a fi cu atât mai mică cu cât vectorul $o$ e mai apropiat de $d$, şi cu atât mai mare cu cât cei doi vectori sunt mai depărtaţi. În plus, se mai consideră un factor de regularizare care împiedică ponderile să devină prea mari în valoare absolută, caz asociat de regulă cu un comportament instabil al reţelei: variaţii mici ale intrării duc la salturi mari în straturile ascunse şi la ieşire.


&emsp;&emsp; În cazul regresiei, neuronii din stratul de ieșire nu au funcție de activare (sau funcția identitate ca funcție de activare), funcția de cost fiind eroarea medie pătratică, iar ieșirea rețelei constă într-un set de valori continue. 

https://skymind.ai/wiki/multilayer-perceptron <br>
https://github.com/lmsasu/cursuri/blob/master/InteligentaArtificiala/curs/InteligentaArtificiala.pdf

## SGDRegressor
Regresie logistică este o modalitate de a modela un sistem discret folosind un funcția logistică (sau o variantă similară). Adică, un sistem cu ieșire are un număr finit de valori posibile. Vă puteți gândi la asta ca la un fel de clasificare algoritm (deși această descriere poate fi periculoasă, deoarece clasificarea este tehnic diferită de regresie), care mapează un set de intrări la un set finit de ieșiri.

Coborârea cu gradient stochastic(SGD) este o variantă a coborâre în gradient (sau coborârea în gradient a lotului) algoritm de optimizare. În loc să utilizeze simultan toate (sau un „lot” de) date de antrenament (poate fi foarte costisitor de calcul / memorie), folosește o aproximare iterativă pentru găsirea minimului unei funcții în spațiul de intrare N-dimensional.

Coborârea cu gradient stochastic poate fi folosit pentru a construi un regresie logistică model similar cu modul în care poate fi folosit pentru a construi un model de regresie liniară. Modelul în sine este independent de algoritmul de optimizare utilizat pentru antrenarea acestuia. In timp ce coborâre gradient stochastic este frecvent utilizat ca algoritm de antrenament, nu este NUMAI opțiune.


Atat estimarea statistica cat si machine learning i-au in considerare problema minimizarii uneu functii obiectiv care are forma sumei:
${\displaystyle Q(w)={\frac {1}{n}}\sum _{i=1}^{n}Q_{i}(w),}{\displaystyle Q(w)={\frac {1}{n}}\sum _{i=1}^{n}Q_{i}(w),}$
unde parametrul  ${\displaystyle w} {\displaystyle Q(w)}$ Q(w) este estimat.Fiecare suma estimata ${\displaystyle Q_{i}}Q_{i}$ este tipic asociata cu   ${\displaystyle i}$  a i-a observatie in dataset(folosit pt antrenare).

Problema sumei minimizare aprea deasemenea ca risc pentru minimizare empirica./ In acest caz, ${\displaystyle Q_{i}(w)}Q_{i}$ (w) este valoarea functiei de loss la  ${\displaystyle i}-lea              exemplu, si {\displaystyle Q(w)}$ Q(w) este riscul empiric.

In stochastic (or "on-line") gradient descent, the true gradient of {\displaystyle Q(w)}Q(w) is approximated by a gradient at a single example:

${\displaystyle w:=w-\eta \nabla Q_{i}(w).}w:=w-\eta \nabla Q_{i}$(w).
In timp ce algoritmul trece prin setul de antrenare,aplica updateul de mai sus pentru fiecare exemplu de antrenare.Cateva treceri pot fi facute peste setul de antrenare pana cand algoritmul converge.Daca se face asta,data poate sa fie amestecate la fiecare pas pentru a preveni ciclurire.Implementarire tipice pot folosi un ritm de invatare adaptiv pentru ca algoritmul sa convearga.

![SGD image](./images/SGD.png)
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html
https://en.wikipedia.org/wiki/Stochastic_gradient_descent
https://ro.ec-europe.org/923532-relationship-logistic-regression-and-stochastic-AJDLNA